# Telecom Megaline Statistical Data Analysis
Project Report by Allentine Paulis

# Table of Contents
* [Project Description](#description)
    * [Description of the plans](#plandescription)
    * [Surf](#surfdesc)
    * [Ultimate](#ultimatedesc)
* [Data](#data)
* [Step 1. Understanding Data](#understanding)
    * [Study the general information : Megaline Calls](#studycalls)
    * [Study the general information : Megaline Internet](#studyinternet)
    * [Study the general information : Megaline Messages](#studymessages)
    * [Study the general information : Megaline Plans](#studyplans)
    * [Study the general information : Megaline Users](#studyusers)
* [Step 2. Data Preprocessing](#preprocessing)
    * [Q1. The number of calls made and minutes used per month](#q1)
    * [Q2. The number of text messages sent per month](#q2)
    * [Q3. The volume of data per month](#q3)
    * [Q4. The monthly revenue from each user](#q4)    
* [Step 3. Exploratory data analysis](#eda)
    * [What factors impact price the most? - Based on overall correlation](#factoroverall)
        * [Heatmap Correlation](#heatmap)
        * [Correlation after outliers removal](#outlierscorr)
        * [Comparison Correlation](#comparecorr)
        * [Correlation with Dummies](#corrdum)
* [Step 4. Hypotheses Testing](#hypotest)        
* [Step 5. Overall conclusion](#allconclusion)

# Project Description <a class="anchor" id="description"></a>

As an analyst for the telecom operator Megaline. The company offers its clients two prepaid plans, Surf and Ultimate. The commercial department wants to know which of the plans brings in more revenue in order to adjust the advertising budget.

We are going to carry out a preliminary analysis of the plans based on a relatively small client selection. We'll have the data on 500 Megaline clients: who the clients are, where they're from, which plan they use, and the number of calls they made and text messages they sent in 2018. The task is to analyze clients' behavior and determine which prepaid plan brings in more revenue.

### Description of the plan  <a class="anchor" id="plandescription"></a>

Note: Megaline rounds seconds up to minutes, and megabytes to gigabytes. For **calls**, each individual call is rounded up: even if the call lasted just one second, it will be counted as one minute. For **web traffic**, individual web sessions are not rounded up. Instead, the total for the month is rounded up. If someone uses 1025 megabytes this month, they will be charged for 2 gigabytes.

### Surf <a class="anchor" id="surfdesc"></a>

1. Monthly charge: $20

2. 500 monthly minutes, 50 texts, and 15 GB of data

3. After exceeding the package limits:
    * 1 minute: 3 cents
    * 1 text message: 3 cents
    * 1 GB of data: $10

### Ultimate <a class="anchor" id="ultimatedesc"></a>

1. Monthly charge: $70

2. 3000 monthly minutes, 1000 text messages, and 30 GB of data

3. After exceeding the package limits:
    * 1 minute: 1 cent
    * 1 text message: 1 cent
    * 1 GB of data: $7

# Data <a class="anchor" id="data"></a>

The `users` table (data on users):
- *user_id* — unique user identifier
- *first_name* — user's name
- *last_name* — user's last name
- *age* — user's age (years)
- *reg_date* — subscription date (dd, mm, yy)
- *churn_date* — the date the user stopped using the service (if the value is missing, the calling plan was being used when this database was extracted)
- *city* — user's city of residence
- *plan* — calling plan name


The `calls` table (data on calls):
- *id* — unique call identifier
- *call_date* — call date
- *duration* — call duration (in minutes)
- *user_id* — the identifier of the user making the call


The `messages` table (data on texts):
- *id* — unique text message identifier
- *message_date* — text message date
- *user_id* — the identifier of the user sending the text


The `internet` table (data on web sessions):
- *id* — unique session identifier
- *mb_used* — the volume of data spent during the session (in megabytes)
- *session_date* — web session date
- *user_id* — user identifier


The `plans` table (data on the plans):
- *plan_name* — calling plan name
- *usd_monthly_fee* — monthly charge in US dollars
- *minutes_included* — monthly minute allowance
- *messages_included* — monthly text allowance
- *mb_per_month_included* — data volume allowance (in megabytes)
- *usd_per_minute* — price per minute after exceeding the package limits (e.g., if the package includes 100 minutes, the 101st minute will be charged)
- *usd_per_message* — price per text after exceeding the package limits
- *usd_per_gb* — price per extra gigabyte of data after exceeding the package limits (1 GB = 1024 megabytes)

## Step 1. Understanding Data  <a class="anchor" id="understanding"></a>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math as mt
from scipy import stats as st
import seaborn as sns

In [2]:
calls = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_calls.csv')
internet = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_internet.csv')
messages = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_messages.csv')
plans = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_plans.csv')
users = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_users.csv')

### Study the general information : Megaline Calls <a class="anchor" id="studycalls"> </a>

In [3]:
calls.head()

,id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66
2,1000_247,1000,2018-12-27,14.48
3,1000_309,1000,2018-12-28,5.76
4,1000_380,1000,2018-12-30,4.22


In [4]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


* call_date data type supposed to be datetime

In [5]:
calls.describe(include='all')

,id,user_id,call_date,duration
count,137735,137735.000000,137735,137735.000000
unique,137735,NaN,351,NaN
top,1292_7,NaN,2018-12-27,NaN
freq,1,NaN,1091,NaN
mean,NaN,1247.658046,NaN,6.745927
std,NaN,139.416268,NaN,5.839241
min,NaN,1000.000000,NaN,0.000000
25%,NaN,1128.000000,NaN,1.290000
50%,NaN,1247.000000,NaN,5.980000
75%,NaN,1365.000000,NaN,10.690000


- There is 0 minimum calls duration, maybe it can be classified as missed call

In [6]:
calls.isna().sum()

id           0
user_id      0
call_date    0
duration     0
dtype: int64

In [7]:
calls.duplicated().sum()

0

In [8]:
calls['id'].nunique() == len(calls)

True

- calls id is unique

In [9]:
calls['user_id'].nunique()

481

- There are 481 unique users in calls

### Study the general information : Megaline Internet <a class="anchor" id="studyinternet"> </a>

In [10]:
internet.head()

,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00
2,1000_379,1000,2018-12-28,660.40
3,1000_413,1000,2018-12-26,270.99
4,1000_442,1000,2018-12-27,880.22


In [11]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


- session_data data types supposed to be datetime

In [12]:
internet.describe(include='all')

,id,user_id,session_date,mb_used
count,104825,104825.000000,104825,104825.000000
unique,104825,NaN,351,NaN
top,1403_357,NaN,2018-12-24,NaN
freq,1,NaN,851,NaN
mean,NaN,1242.496361,NaN,366.713701
std,NaN,142.053913,NaN,277.170542
min,NaN,1000.000000,NaN,0.000000
25%,NaN,1122.000000,NaN,136.080000
50%,NaN,1236.000000,NaN,343.980000
75%,NaN,1367.000000,NaN,554.610000


- There is 0 mb_used, maybe the users use WiFi not this regular internet connection.

In [13]:
internet.isna().sum()

id              0
user_id         0
session_date    0
mb_used         0
dtype: int64

In [14]:
internet.duplicated().sum()

0

In [15]:
internet['id'].nunique() == len(internet)

True

- Internet id is unique

In [16]:
internet['user_id'].nunique()

489

- There is 489 unique users using internet

### Study the general information : Megaline Messages <a class="anchor" id="studymessages"> </a>

In [17]:
messages.head()

,id,user_id,message_date
0,1000_125,1000,2018-12-27
1,1000_160,1000,2018-12-31
2,1000_223,1000,2018-12-31
3,1000_251,1000,2018-12-27
4,1000_255,1000,2018-12-26


In [18]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


- message_date data types supposed to be datetime

In [19]:
messages.describe(include='all')

,id,user_id,message_date
count,76051,76051.000000,76051
unique,76051,NaN,351
top,1119_1056,NaN,2018-12-28
freq,1,NaN,702
mean,NaN,1245.972768,NaN
std,NaN,139.843635,NaN
min,NaN,1000.000000,NaN
25%,NaN,1123.000000,NaN
50%,NaN,1251.000000,NaN
75%,NaN,1362.000000,NaN


In [20]:
messages.isna().sum()

id              0
user_id         0
message_date    0
dtype: int64

In [21]:
messages.duplicated().sum()

0

In [22]:
messages['id'].nunique() == len(messages)

True

- There is unique id in messages

In [23]:
messages['user_id'].nunique()

402

- There is 402 unique users using messages

### Study the general information : Megaline Plans <a class="anchor" id="studyplans"> </a>

In [24]:
plans.head()

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


In [25]:
plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 256.0+ bytes


### Study the general information : Megaline Users <a class="anchor" id="studyusers"> </a>

In [26]:
users.head()

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN


In [27]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


- reg_date data types supposed to be datetime

In [28]:
users.isna().sum()

user_id         0
first_name      0
last_name       0
age             0
city            0
reg_date        0
plan            0
churn_date    466
dtype: int64

In [29]:
users.duplicated().sum()

0

In [30]:
users['churn_date'].isna().sum()/ len(users) * 100

93.2

In [31]:
(len(users) - users['churn_date'].isna().sum())/len(users) * 100

6.800000000000001

- churn has wrong data type and supposed to be datetime
- 93.2% not churned. which means 6.8% customers churned

In [32]:
users['user_id'].nunique() == len(users)

True

- user id is unique and there are 500 unique user id

### Conclusion

- call_date has wrong data type and supposed to be datetime
- There is 0 minimum calls duration, maybe it can be classified as missed call if duration < 0.1
- There are 481 unique users in calls


- session_data has wrong data type and supposed to be datetime
- There is 0 mb_used, it's still possible, maybe the users use WiFi not this regular internet connection.
- There is 489 unique users using internet


- message_date data types supposed to be datetime
- There is 402 unique users using messages


- reg_date has wrong data type and supposed to be datetime
- churn_date has wrong data type and supposed to be datetime
- 6.8% customers churned. 93.2% not churned and null.
- user id is unique and there are total 500 unique user id, but not all using services. 


## Step 2. Data Preprocessing  <a class="anchor" id="preprocessing"></a>

Convert the data to the necessary types
- Find and eliminate errors in the data
- Explain what errors you found and how you removed them.


For each user, find:
1. The number of calls made and minutes used per month
2. The number of text messages sent per month
3. The volume of data per month
4. The monthly revenue from each user (subtract the free package limit from the total number of calls, text messages, and data; multiply the result by the calling plan value; add the monthly charge depending on the calling plan)

- Fix datetime data types

In [49]:
calls['call_date'] = pd.to_datetime(calls['call_date'])
internet['session_date'] = pd.to_datetime(internet['session_date'])
messages['message_date'] = pd.to_datetime(messages['message_date'])
users['reg_date'] = pd.to_datetime(users['reg_date'])
users['churn_date'] = pd.to_datetime(users['churn_date'])

In [50]:
calls.head()

,id,user_id,call_date,duration,day,month,year
0,100093,1000,2018-12-27,8.52,27,12,2018
1,1000145,1000,2018-12-27,13.66,27,12,2018
2,1000247,1000,2018-12-27,14.48,27,12,2018
3,1000309,1000,2018-12-28,5.76,28,12,2018
4,1000380,1000,2018-12-30,4.22,30,12,2018


In [51]:
calls['day'] = calls['call_date'].dt.day_name()
calls['month'] = calls['call_date'].dt.month
calls['year'] = calls['call_date'].dt.year

In [52]:
calls.head()

,id,user_id,call_date,duration,day,month,year
0,100093,1000,2018-12-27,8.52,Thursday,12,2018
1,1000145,1000,2018-12-27,13.66,Thursday,12,2018
2,1000247,1000,2018-12-27,14.48,Thursday,12,2018
3,1000309,1000,2018-12-28,5.76,Friday,12,2018
4,1000380,1000,2018-12-30,4.22,Sunday,12,2018


In [53]:
calls.dtypes

id                    int64
user_id               int64
call_date    datetime64[ns]
duration            float64
day                  object
month                 int64
year                  int64
dtype: object

- Fix id data type to int

In [54]:
calls['id'] = calls['id'].astype('int64')
internet['id'] = internet['id'].astype('int64')
messages['id'] = messages['id'].astype('int64')

In [55]:
calls.head()

,id,user_id,call_date,duration,day,month,year
0,100093,1000,2018-12-27,8.52,Thursday,12,2018
1,1000145,1000,2018-12-27,13.66,Thursday,12,2018
2,1000247,1000,2018-12-27,14.48,Thursday,12,2018
3,1000309,1000,2018-12-28,5.76,Friday,12,2018
4,1000380,1000,2018-12-30,4.22,Sunday,12,2018


In [56]:
internet.head()

,id,user_id,session_date,mb_used
0,100013,1000,2018-12-29,89.86
1,1000204,1000,2018-12-31,0.00
2,1000379,1000,2018-12-28,660.40
3,1000413,1000,2018-12-26,270.99
4,1000442,1000,2018-12-27,880.22


In [57]:
messages.head()

,id,user_id,message_date
0,1000125,1000,2018-12-27
1,1000160,1000,2018-12-31
2,1000223,1000,2018-12-31
3,1000251,1000,2018-12-27
4,1000255,1000,2018-12-26


- make string lower case

In [58]:
users['city'] = users['city'].str.lower()
users['first_name'] = users['first_name'].str.lower()
users['last_name'] = users['last_name'].str.lower()

In [59]:
users.sample(7)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
106,1106,eddy,garrison,47,"new york-newark-jersey city, ny-nj-pa msa",2018-05-10,surf,2018-11-14
429,1429,arianna,rowe,36,"new york-newark-jersey city, ny-nj-pa msa",2018-12-22,surf,NaT
294,1294,demetrius,russo,67,"new york-newark-jersey city, ny-nj-pa msa",2018-04-19,surf,NaT
433,1433,lyndon,morrison,29,"philadelphia-camden-wilmington, pa-nj-de-md msa",2018-08-29,ultimate,NaT
483,1483,miguel,farrell,35,"chicago-naperville-elgin, il-in-wi msa",2018-04-25,surf,NaT
138,1138,arlie,higgins,66,"riverside-san bernardino-ontario, ca msa",2018-02-06,surf,NaT
1,1001,mickey,wilkerson,28,"seattle-tacoma-bellevue, wa msa",2018-08-13,surf,NaT


- Rounded up calls duration and internet mb_used

In [60]:
calls['duration'] = calls['duration'].apply(np.ceil)
internet['mb_used'] = internet['mb_used'].apply(np.ceil)

### Q1. The number of calls made and minutes used per month <a class="anchor" id="q1"></a>

In [67]:
calls_usage = calls.groupby(['user_id','month']).agg({'duration':'sum','id':'count'}).rename(columns={'duration':'Minutes Used per month','id':'Calls Made'})
calls_usage

Minutes Used per month  Calls Made
user_id month                                    
1000    12                      124.0          16
1001    8                       182.0          27
        9                       315.0          49
        10                      393.0          65
        11                      426.0          64
...                               ...         ...
1498    12                      339.0          39
1499    9                       346.0          41
        10                      385.0          53
        11                      308.0          45
        12                      496.0          65

[2258 rows x 2 columns]

### Q2. The number of text messages sent per month <a class="anchor" id="q2"></a>

In [63]:
messages['month'] = pd.DatetimeIndex(messages['message_date']).month

In [68]:
messages_usage = messages.groupby(['user_id','month']).agg({'id':'count'}).rename(columns={'id':'SMS per month'})
messages_usage

SMS per month
user_id month               
1000    12                11
1001    8                 30
        9                 44
        10                53
        11                36
...                      ...
1496    9                 21
        10                18
        11                13
        12                11
1497    12                50

[1806 rows x 1 columns]

### Q3. The volume of data per month <a class="anchor" id="q3"></a>

In [71]:
internet['month'] = pd.DatetimeIndex(internet['session_date']).month

In [72]:
internet_usage = internet.groupby(['user_id','month']).agg({'mb_used':'sum'}).rename(columns={'mb_used':'Data used per month in mb'})
internet_usage

Data used per month in mb
user_id month                           
1000    12                        1903.0
1001    8                         6929.0
        9                        13333.0
        10                       22356.0
        11                       18524.0
...                                  ...
1498    12                       23160.0
1499    9                        13004.0
        10                       19515.0
        11                       16834.0
        12                       22091.0

[2277 rows x 1 columns]

### Q4. The monthly revenue from each user <a class="anchor" id="q4"></a>
(subtract the free package limit from the total number of calls, text messages, and data; multiply the result by the calling plan value; add the monthly charge depending on the calling plan) 

Steps:
- Combine all previous dataframes to one
- Find per user, the plan they have
- check for each parameter if they used more than they should and calculate it

In [76]:
user_plan = users[['user_id','plan']]
user_plan

,user_id,plan
0,1000,ultimate
1,1001,surf
2,1002,surf
3,1003,surf
4,1004,surf
...,...,...
495,1495,surf
496,1496,surf
497,1497,ultimate
498,1498,surf


In [94]:
users_usage = pd.concat([calls_usage, messages_usage, internet_usage], axis=1).reset_index().merge(user_plan, on='user_id')
users_usage.columns= ['user_id','month','minutes','calls','sms','mb_used','plan']

In [95]:
users_usage

,user_id,month,minutes,calls,sms,mb_used,plan
0,1000,12,124.0,16.0,11.0,1903.0,ultimate
1,1001,8,182.0,27.0,30.0,6929.0,surf
2,1001,9,315.0,49.0,44.0,13333.0,surf
3,1001,10,393.0,65.0,53.0,22356.0,surf
4,1001,11,426.0,64.0,36.0,18524.0,surf
...,...,...,...,...,...,...,...
2288,1498,12,339.0,39.0,NaN,23160.0,surf
2289,1499,9,346.0,41.0,NaN,13004.0,surf
2290,1499,10,385.0,53.0,NaN,19515.0,surf
2291,1499,11,308.0,45.0,NaN,16834.0,surf


In [167]:
users_usage.isnull().sum()

user_id      0
month        0
minutes     35
calls       35
sms        487
mb_used     16
plan         0
revenue      0
dtype: int64

In [168]:
# FIll missing values with 0
users_usage = users_usage.fillna(0)

In [169]:
plans.head()

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


In [170]:
plans.query('plan_name == "ultimate"').loc[:,'messages_included'].values[0]

1000

In [171]:
#initialize surf plan
surf_price = plans.query('plan_name =="surf"').loc[:,'usd_monthly_pay'].values[0]
surf_messages = plans.query('plan_name =="surf"').loc[:,'messages_included'].values[0]
surf_data = plans.query('plan_name =="surf"').loc[:,'mb_per_month_included'].values[0]
surf_minutes = plans.query('plan_name =="surf"').loc[:,'minutes_included'].values[0]
surf_price_for_mb = plans.query('plan_name =="surf"').loc[:,'usd_per_gb'].values[0] / 1000
surf_price_for_message =  plans.query('plan_name =="surf"').loc[:,'usd_per_message'].values[0]
surf_price_for_minute = plans.query('plan_name =="surf"').loc[:,'usd_per_minute'].values[0]

#initialize ultimate plan
ultimate_price = plans.query('plan_name =="ultimate"').loc[:,'usd_monthly_pay'].values[0] 
ultimate_messages = plans.query('plan_name == "ultimate"').loc[:,'messages_included'].values[0]
ultimate_data = plans.query('plan_name =="ultimate"').loc[:,'mb_per_month_included'].values[0]
ultimate_minutes = plans.query('plan_name =="ultimate"').loc[:,'minutes_included'].values[0]
ultimate_price_for_mb = plans.query('plan_name =="ultimate"').loc[:,'usd_per_gb'].values[0] / 1000
ultimate_price_for_message = plans.query('plan_name =="ultimate"').loc[:,'usd_per_message'].values[0]
ultimate_price_for_minute = plans.query('plan_name =="ultimate"').loc[:,'usd_per_minute'].values[0]

In [172]:
print(f"surf price : {surf_price} usd monthly")
print(f"surf messages : {surf_message} messages")
print(f"surf data : {surf_data} mb per month")
print(f"surf minutes : {surf_minutes} minutes")
print(f"surf price for mb :{surf_price_for_mb} usd") 
print(f"surf price for message :{surf_price_for_mb} usd")
print(f"surf price for minute :{surf_price_for_mb} usd")
print("\n")
print(f"ultimate price : {ultimate_price} usd monthly")
print(f"ultimate messages : {ultimate_message} messages")
print(f"ultimate data : {ultimate_data} mb per month")
print(f"ultimate minutes : {ultimate_minutes} minutes")
print(f"ultimate price for mb :{ultimate_price_for_mb} usd") 
print(f"ultimate price for message :{ultimate_price_for_message} usd")
print(f"ultimate price for minute :{ultimate_price_for_minute} usd")

surf price : 20 usd monthly
surf messages : 50 messages
surf data : 15360 mb per month
surf minutes : 500 minutes
surf price for mb :0.01 usd
surf price for message :0.01 usd
surf price for minute :0.01 usd


ultimate price : 70 usd monthly
ultimate messages : 1000 messages
ultimate data : 30720 mb per month
ultimate minutes : 3000 minutes
ultimate price for mb :0.007 usd
ultimate price for message :0.01 usd
ultimate price for minute :0.01 usd


In [173]:
def revenue(row):
    mb_used = row['mb_used']
    plan = row['plan']
    minutes = row['minutes']
    messages = row['sms']
    total_mb = 0
    total_messages = 0 
    total_minutes = 0
    if plan == 'surf':
        if mb_used > surf_data:
            total_mb = (mb_used - surf_data) * surf_price_for_mb
        if minutes > surf_minutes:
            total_minutes = (minutes - surf_minutes) * surf_price_for_minute
        if messages > surf_messages:
            total_messages = (messages - surf_messages) * surf_price_for_message
        
        return total_mb + total_minutes + total_messages + surf_price

    else: # ultimate plan
        if mb_used > ultimate_data:
            total_mb = (mb_used - ultimate_data) * ultimate_price_for_mb
        if minutes > ultimate_minutes:
            total_minutes = (minutes - ultimate_minutes) * ultimate_price_for_minute
        if messages > ultimate_messages:
            total_messages = (messages - ultimate_message) * ultimate_price_for_message
        return total_mb + total_minutes + total_messages + ultimate_price

In [174]:
users_usage['revenue'] = users_usage.apply(revenue, axis=1)

In [175]:
users_usage.sample(10)

,user_id,month,minutes,calls,sms,mb_used,plan,revenue
736,1155,4,294.0,36.0,49.0,18550.0,ultimate,70.00
990,1206,9,423.0,66.0,35.0,26935.0,surf,135.75
2081,1449,11,291.0,39.0,0.0,15848.0,surf,24.88
589,1125,12,517.0,65.0,54.0,22725.0,surf,94.28
1353,1291,10,681.0,91.0,14.0,21297.0,surf,84.80
2132,1464,12,629.0,93.0,27.0,18710.0,surf,57.37
601,1127,10,168.0,27.0,12.0,24350.0,surf,109.90
1641,1357,5,16.0,2.0,1.0,1101.0,surf,20.00
1233,1259,10,99.0,15.0,41.0,1956.0,surf,20.00
966,1201,8,627.0,81.0,6.0,35538.0,surf,225.59


In [176]:
# Most profitable users
users_usage.iloc[users_usage['revenue'].idxmax(),:]

user_id      1379
month          12
minutes      1103
calls         144
sms           126
mb_used     70988
plan         surf
revenue    596.65
Name: 1756, dtype: object

In [177]:
# Less profitable users
users_usage.iloc[users_usage['revenue'].idxmin(),:]

user_id    1001
month         8
minutes     182
calls        27
sms          30
mb_used    6929
plan       surf
revenue      20
Name: 1, dtype: object

In [186]:
# All users with less profitable revenue
users_usage[users_usage['revenue']==20]

,user_id,month,minutes,calls,sms,mb_used,plan,revenue
1,1001,8,182.0,27.0,30.0,6929.0,surf,20.0
2,1001,9,315.0,49.0,44.0,13333.0,surf,20.0
6,1002,10,59.0,11.0,15.0,6559.0,surf,20.0
8,1002,12,384.0,47.0,41.0,14416.0,surf,20.0
10,1004,5,193.0,21.0,7.0,6554.0,surf,20.0
...,...,...,...,...,...,...,...,...
2272,1496,8,122.0,18.0,2.0,8616.0,surf,20.0
2274,1496,10,317.0,46.0,18.0,14308.0,surf,20.0
2275,1496,11,308.0,39.0,13.0,8561.0,surf,20.0
2282,1498,6,238.0,37.0,0.0,14825.0,surf,20.0


In [178]:
# Total Revenue
users_usage['revenue'].sum()

144261.193

### Conclusion

- All date data type are fixed to datetime
- All id data type are fixed to int
- Change str to lower case
- Using groupby, we got number of calls made and minutes used per month, number of text messages sent per month,  volume of data per month
- To get the montly revenue per each user, we combine all previous dataframes to one, to find per user, the plan they have and check for each parameter if they used more than they should and calculate it by subtract the free package limit from the total number of calls, text messages, and data; multiply the result by the calling plan value; add the monthly charge depending on the calling plan
- Most profitable user is in surf plan for user_id 1379 and month 12 with revenue 596.65
- Total revenue is 144261.19 USD

## Step 3. Exploratory Data Analysis <a class="anchor" id="eda"></a>

- Describe the customers' behavior. 
- Find the minutes, texts, and volume of data the users of each plan require per month. 
- Calculate the mean, variance, and standard deviation. 
- Plot histograms. 
- Describe the distributions.

In [190]:
users['plan'].value_counts(normalize=True) * 100

surf        67.8
ultimate    32.2
Name: plan, dtype: float64

- 67.8% customers choose Surf plan. 32.2% customers choose Ultimate plan

In [197]:
# Find the minutes, texts, and volume of data the users of each plan require per month.
status = users_usage.groupby(['plan','month']).agg({'minutes':'sum','sms':'sum','mb_used':'sum'})
status

minutes      sms    mb_used
plan     month                              
surf     1         406.0     21.0     9762.0
         2        2673.0    108.0   109736.0
         3        7590.0    351.0   307304.0
         4       17577.0    870.0   599910.0
         5       30768.0   1849.0  1074400.0
         6       41836.0   2454.0  1485998.0
         7       54448.0   3271.0  2033185.0
         8       66438.0   4662.0  2724008.0
         9       80360.0   5968.0  3222517.0
         10     101846.0   8020.0  4107618.0
         11     119801.0   9165.0  4629467.0
         12     150680.0  12275.0  5772904.0
ultimate 1         774.0     62.0    27705.0
         2        2822.0    151.0   120042.0
         3        3651.0    243.0   220083.0
         4        7074.0    463.0   338950.0
         5       11781.0    931.0   482669.0
         6       17435.0   1379.0   721690.0
         7       25197.0   1937.0   965457.0
         8       29922.0   2732.0  1266277.0
         9       37626.0   3259.0  1461088.0
         10      47778.0   4181.0  1869118.0
         11      56540.0   4903.0  2165809.0
         12      69324.0   6796.0  2770055.0

## Step 4. Hypotheses Testing <a class="anchor" id="hypotest"></a>

## Step 5. Overall conclusion <a class="anchor" id="allconclusion"></a>